In [4]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://ww，w.backtrader.com/docu/inddev.html
class RSI_Oversold(bt.Indicator):
    lines = ('rsi_os_30',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.rsi_os_30 = bt.indicators.RSI(self.data, period=6) < self.p.level

class RSI_Overbought(bt.Indicator):
    lines = ('rsi_ob_70',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.rsi_ob_70 = bt.indicators.RSI(self.data, period=6) > self.p.level

class LRSI_Oversold(bt.Indicator):
    lines = ('lrsi_os_03',)
    
    params = (
        ('level', .3),    
    )

    def __init__(self):
        self.lines.lrsi_os_03 = bt.indicators.LRSI(self.data, period=6) < self.p.level

class LRSI_Overbought(bt.Indicator):
    lines = ('lrsi_ob_07',)
    
    params = (
        ('level', .7),    
    )

    def __init__(self):
        self.lines.lrsi_ob_07 = bt.indicators.LRSI(self.data, period=6) > self.p.level

class TSI_Oversold(bt.Indicator):
    lines = ('tsi_os',)
    
    params = (
        ('level', 0),    
    )

    def __init__(self):
        self.lines.tsi_os = bt.indicators.TSI(self.data, period1=10, period2=3) > self.p.level

class TSI_Overbought(bt.Indicator):
    lines = ('tsi_ob',)
    
    params = (
        ('level', 0),    
    )

    def __init__(self):
        self.lines.tsi_ob = bt.indicators.TSI(self.data, period1=10, period2=3) < self.p.level

class UO_Oversold(bt.Indicator):
    lines = ('uo_os_30',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.uo_os_30 = bt.indicators.UltimateOscillator(self.data).lines.uo < self.p.level

class UO_Overbought(bt.Indicator):
    lines = ('uo_ob_60',)
    
    params = (
        ('level', 60),    
    )

    def __init__(self):
        self.lines.uo_ob_60 = bt.indicators.UltimateOscillator(self.data).lines.uo > self.p.level

class AO_Oversold(bt.Indicator):
    lines = ('ao_os',)
    
    params = (
        ('level', 0),    
    )

    def __init__(self):
        self.lines.ao_os = bt.indicators.AwesomeOscillator(self.data).lines.ao > self.p.level

class AO_Overbought(bt.Indicator):
    lines = ('ao_ob',)
    
    params = (
        ('level', 0),    
    )

    def __init__(self):
        self.lines.ao_ob = bt.indicators.AwesomeOscillator(self.data).lines.ao < self.p.level

class CCI_Oversold(bt.Indicator):
    lines = ('cci_os',)
    
    params = (
        ('level', -80),    
    )

    def __init__(self):
        self.lines.cci_os = bt.indicators.CCI(self.data, period=6).lines.cci < self.p.level

class CCI_Overbought(bt.Indicator):
    lines = ('cci_ob',)
    
    params = (
        ('level', 80),    
    )

    def __init__(self):
        self.lines.cci_ob = bt.indicators.CCI(self.data, period=6).lines.cci > self.p.level

class WR_Oversold(bt.Indicator):
    lines = ('wr_os',)
    
    params = (
        ('level', -70),    
    )

    def __init__(self):
        self.lines.wr_os = bt.indicators.WilliamsR(self.data, period=6).lines.percR < self.p.level

class WR_Overbought(bt.Indicator):
    lines = ('wr_ob',)
    
    params = (
        ('level', -30),    
    )

    def __init__(self):
        self.lines.wr_ob = bt.indicators.WilliamsR(self.data, period=6).lines.percR > self.p.level
        
# Create a Strategy
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ("symbols", None),
        ("max_days_in_trade", 5)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
        
        # Keep a reference to the close and forward lagged close prices as well as returns
        self.stock = self.datas[symbols.index("stockpx")]
        self.stock_px = self.stock.close
        self.stock_px_enter = self.datas[symbols.index("stockpx_lag_1")].close
        self.stock_px_exit = self.datas[symbols.index("stockpx_lag_%d" % self.p.max_days_in_trade)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.rsi_oversold = RSI_Oversold(self.stock_px)
        self.rsi_overbought = RSI_Overbought(self.stock_px)
        self.lrsi_oversold = LRSI_Oversold(self.stock_px)
        self.lrsi_overbought = LRSI_Overbought(self.stock_px)
        self.tsi_oversold = TSI_Oversold(self.stock_px)
        self.tsi_overbought = TSI_Overbought(self.stock_px)
        # pass in stock not just stock_px as indicator also requires high/low prices
        self.ao_oversold = AO_Oversold(self.stock)
        self.ao_overbought = AO_Overbought(self.stock)
        self.uo_oversold = UO_Oversold(self.stock)
        self.uo_overbought = UO_Overbought(self.stock)
        self.cci_oversold = CCI_Oversold(self.stock)
        self.cci_overbought = CCI_Overbought(self.stock)
        self.wr_oversold = WR_Oversold(self.stock)
        self.wr_overbought = WR_Overbought(self.stock)
        
                
        self.in_trade = False
        self.days_in_trade = 0
        
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f' % self.stock_px[0])
        
        size = self.position.size
        
        if(not size):
            if self.rsi_oversold[0] > 0:
#             if self.bollinger_breakdown[0] > 0:
                # replicate straddle payout less premium
#                 print("Indicator triggered...entering position")
#                 print('Exit price %.2f enter price %.2f' % (self.stock_px_exit[0], self.stock_px_enter[0]))
                if self.stock_px_exit[0] > self.stock_px_enter[0]:
#                     print("Exit price will be higher than enter price. Buy!")
                    self.buy(exectype=bt.Order.Close)
                else:
#                     print("Exit price will be lower than enter price. Sell!")
                    self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0

        
        
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    asset = 'YHOO'
    n_lags = 6 
    
    # Read in stock and vol data 
    datapath_vol = os.path.join('../../../../datas/%s.csv' % asset) 
#     datapath_stock = os.path.join('../../../../datas/%s-2010-2018.csv' % asset)
    datapath_stock = os.path.join('../../../../datas/yhoo-1996-2015.txt')
    data_vol = pd.read_csv(datapath_vol, parse_dates=True,index_col=0)
    data_stock = pd.read_csv(datapath_stock, parse_dates=True,index_col=0)
    
    # Lag and merge data
    for n in np.arange(1, n_lags):
        data_stock['stockpx_lag_%d' % n ] = data_stock['Close'].shift(-n)  
    data_stock = data_stock.dropna()
    
    mergedDf = pd.concat([data_vol,data_stock], axis=1,join='inner')
    data_vol = data_vol.loc[data_vol.index & mergedDf.index]
    data_stock = data_stock.loc[data_stock.index & mergedDf.index]
    
    # Add stock price data
    cerebro.adddata(bt.feeds.PandasData(dataname=data_stock[['Close', 'High', 'Low', 'Volume']]), name='stockpx')
    
    # Add lagged stock prices to be able to replicate straddle payout
    symbol_cols = [ 'stockpx_lag_%d' % n for n in np.arange(1, n_lags)] 
    for i, symbol in enumerate(symbol_cols):
        data = data_stock[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add straddle premium
    data = data_vol[['10dORHV']]
    data.columns = ["Close"]
    cerebro.adddata(bt.feeds.PandasData(dataname=data), name='10dORHV')
    
    # Add a strategy
    symbol_cols = ['stockpx'] + symbol_cols + ['10dORHV']
    cerebro.addstrategy(TestStrategy, symbols=symbol_cols)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='YHOO-indicators.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100061.40
